# 模型类

In [18]:
from abc import abstractmethod, ABC
import numpy as np

## Foundation

### Tensor

In [19]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = None
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    def size(self):
        return self.data.shape[-1]

    def __str__(self):
        return str(self.data)

### Base Dataset

In [20]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size(), Tensor(self.labels).size()

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

    @abstractmethod
    def estimate(self, predictions):
        pass

### Base Layer

In [21]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def parameters(self):
        return []

    def __str__(self):
        return ''

### Base Loss Function

In [22]:
class Loss(ABC):

    @abstractmethod
    def __call__(self, p: Tensor, y: Tensor):
        pass

### Base Optimizer

In [23]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    @abstractmethod
    def backward(self):
        pass

### Base Model

In [24]:
class Model(ABC):

    def __init__(self, layer, loss, optimizer):
        self.layer = layer
        self.loss = loss
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## Data

### MLP Dataset

In [25]:
class MLPDataset(Dataset):

    def load(self):
        self.train_features = [[22.5, 72.0],
                               [31.4, 45.0],
                               [19.8, 85.0],
                               [27.6, 63]]

        self.train_labels = [[95],
                             [210],
                             [70],
                             [155]]

        self.test_features = [[28.1, 58.0]]

        self.test_labels = [[165]]

    def estimate(self, predictions):
        return ((predictions.data - self.labels) ** 2).mean()

## Model

### Linear Layer

In [26]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad = p.grad.T @ x.data / len(x.data)
            self.bias.grad = np.sum(p.grad, axis=0) / len(x.data)

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias}
        return p

    def parameters(self):
        return [self.weight, self.bias]

    def __str__(self):
        return f'weight: {self.weight}\nbias: {self.bias}'

### MSE Loss Function

In [27]:
class MSELoss(Loss):

    def __call__(self, p: Tensor, y: Tensor):
        mse = Tensor(((y.data - p.data) ** 2).mean())

        def gradient_fn():
            p.grad = -2 * (y.data - p.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### SGD Optimizer

In [28]:
class SGDOptimizer(Optimizer):

    def backward(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### Neural Network Model

In [29]:
class NNModel(Model):

    def train(self, dataset, epochs):
        dataset.train()

        for epoch in range(epochs):
            for i in range(len(dataset)):
                features, labels = dataset[i]

                predictions = self.layer(features)
                error = self.loss(predictions,
                                  labels)

                error.backward()
                self.optimizer.backward()

    def test(self, dataset):
        dataset.eval()

        features, labels = dataset.items()
        return self.layer(features)

## Configuration

### Learning Rate

In [30]:
LEARNING_RATE = 0.00001

### Batch

In [31]:
BATCH_SIZE = 2

### Epoch

In [32]:
EPOCHS = 1000

## Training

### Iteration Training

In [33]:
dataset = MLPDataset(BATCH_SIZE)
layer = Linear(dataset.shape()[0],
               dataset.shape()[1])
loss = MSELoss()
optimizer = SGDOptimizer(layer.parameters(),
                         lr=LEARNING_RATE)

model = NNModel(layer,
                loss,
                optimizer)
model.train(dataset,
            EPOCHS)

print(layer)

weight: [[ 8.04709172 -1.08194575]]
bias: [0.15285452]


## Testing

### Estimating

In [34]:
predictions = model.test(dataset)

print(f'prediction: {predictions}')
print(f'error: {dataset.estimate(predictions)}')

prediction: [[163.52327795]]
error: 2.1807080003282495
